In [ ]:
import os
import torch
from torch.utils.data import random_split
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data.dataloader import DataLoader
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from resnet18 import ResNet18, BasicBlock

## 数据预处理

In [ ]:
data_dir = '../data/data'

In [ ]:
dataset = ImageFolder(data_dir+'/train', transform=transforms.Compose([transforms.ToTensor()]))
classes = os.listdir(data_dir + "/train")

In [ ]:
random_seed = 1234
torch.manual_seed(random_seed)

val_size = 8202 # 10% of the total size
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds)
val_data_size = len(val_ds)

In [ ]:
batch_size= 128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=0, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=0,pin_memory=True)

## 构建网络

In [ ]:
device=torch.device('cuda'if torch.cuda.is_available() else "cpu")
net=ResNet18(BasicBlock)
net.to(device)
writer = SummaryWriter("logs")
writer.add_graph(net, input_to_model=torch.rand(1,3,64,64).to(device))

## 选择优化器和学习率



In [ ]:
lr=0.0001
optim=torch.optim.Adam(net.parameters(),lr=lr)
sculer=torch.optim.lr_scheduler.StepLR(optim,step_size=1)


## 训练



In [ ]:
from ultis import plot_preds

total_train_step = 1
total_val_step = 1
epochs=6 # 循环次数
for epoch in range(epochs):
    total_train=0
    for data in train_dl:
        img,label=data
        img =img.to(device)
        label=label.to(device)
        optim.zero_grad()
        output=net(img)
        train_loss=nn.CrossEntropyLoss()(output,label).to(device)
        train_loss.backward()
        optim.step()
        total_train+=train_loss
        total_train_step += 1
        if total_train_step % 100 == 0:
            writer.add_scalar("train_loss", total_train, total_train_step)
    sculer.step()
    total_test=0
    total_accuracy=0
    for data in val_dl:
        img,label =data
        with torch.no_grad():
            img=img.to(device)
            label=label.to(device)
            out=net(img)
            test_loss=nn.CrossEntropyLoss()(out,label).to(device)
            total_test+=test_loss
            accuracy=(out.argmax(1)==label).sum()
            total_accuracy+=accuracy
    print("训练集上的损失：{}".format(total_train))
    print("验证集上的损失：{}".format(total_test))
    print("验证集上的精度：{:.1%}".format(total_accuracy/val_data_size))
    writer.add_scalar("val_loss", total_test, total_val_step)
    writer.add_scalar("val_accuracy", total_accuracy/val_data_size, total_val_step)
    writer.add_scalar("learnig_rate", optim.param_groups[0]["lr"], total_val_step)
    fig = plot_preds(net, "../plot/", transforms.ToTensor(), device="cuda")
    writer.add_figure("prediction vs real", fig, epoch+1)
    total_train_step += 1
    total_val_step += 1
    torch.save(net,"ResNet.{}.pth".format(epoch+1))
    print("模型已保存")
writer.close()
